In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math

In [2]:
mnist = tf.keras.datasets.mnist

In [4]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 1s 0us/step

In [6]:
sess = tf.InteractiveSession()

In [9]:
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784], name = 'x-input')
    y_ = tf.placeholder(tf.int64, [None], name = 'y-input')
with tf.name_scope('input-reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', image_shaped_input, 10)

In [13]:
def weight_variable(shape):
    weight = tf.Variable(tf.truncated_normal(shape, stddev = 0.1))
    return weight
def bias_variable(shape):
    bias = tf.Variable(tf.constant(0.1, shape=shape))
    return bias
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('hist', var)

In [16]:
def nn_layer(input_tensor, input_dim, output_dim, layer_name, activation=tf.nn.relu):
    with tf.name_scope(layer_name):
        with tf.name_scope('weight'):
            weight = weight_variable([input_dim, output_dim])
            variable_summaries(weight)
        with tf.name_scope('bias'):
            bias = bias_variable([output_dim])
            variable_summaries(bias)
        with tf.name_scope('Wx_plus_b'):
            preactivate = tf.matmul(input_tensor, weight) + bias
            tf.summary.histogram('preactivate', preactivate)
        
        activations = activation(preactivate, name = 'activation')
        tf.summary.histogram('activation', activations)
    return activations

In [17]:
hidden1 = nn_layer(x, 784, 500, 'layer1')

with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    tf.summary.scalar('keep_prob', keep_prob)
    dropped = tf.nn.dropout(hidden1, keep_prob)

y = nn_layer(dropped, 500, 10, 'layer2', activation=tf.identity)

In [ ]:
with tf.name_scope('cross_entropy'):
    with tf.name_scope('total'):
        cross_entropy = tf.losses.sparse_softmax_cross_entropy(label = y_, logits = y)